# Model training

In progress...

In [ ]:
from config import *
import torch
from models_local import ResidualBlock
from models_local import Model
import torch.nn as nn
import torch.optim as optim

from OFDM_SDR_Functions_torch import *
import numpy as np
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.pdsch_iq = []
        self.pilot_iq = []
        self.labels = []
        
    def __len__(self):
        return len(self.pdsch_iq)
    
    def __getitem__(self, index):
        x1 = self.pdsch_iq[index]
        x2 = self.pilot_iq[index]
        y = self.labels[index]
        return x1, x2, y
    
    def add_item(self, new_pdsch_iq, new_pilot_iq, new_label):
        self.pdsch_iq.append(new_pdsch_iq)
        self.pilot_iq.append(new_pilot_iq)
        self.labels.append(new_label)
        
dataset = CustomDataset()

In [ ]:
dataset = torch.load('ofdm_dataset.pth')

In [ ]:
###################### DeepRX PyTorch #############################
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, num_conv_channels):
        super(ResidualBlock, self).__init__()

        # Layer normalization is done over the last three dimensions: time, frequency, conv 'channels'
        self.layer_norm_1 = nn.LayerNorm((num_conv_channels, None), elementwise_affine=False)
        self.conv_1 = nn.Conv2d(num_conv_channels, num_conv_channels, kernel_size=3, padding=1)

        # Layer normalization is done over the last three dimensions: time, frequency, conv 'channels'
        self.layer_norm_2 = nn.LayerNorm((num_conv_channels, None), elementwise_affine=False)
        self.conv_2 = nn.Conv2d(num_conv_channels, num_conv_channels, kernel_size=3, padding=1)

    def forward(self, inputs):
        z = self.layer_norm_1(inputs)
        z = F.relu(z)
        z = self.conv_1(z)
        z = self.layer_norm_2(z)
        z = F.relu(z)
        z = self.conv_2(z)

        # Skip connection
        z = z + inputs

        return z

###############################################


class Model(nn.Module):
    def __init__(self, num_conv_channels, num_bits_per_symbol):
        super(Model, self).__init__()

        # Input convolution
        self.input_conv = nn.Conv2d(num_conv_channels, num_conv_channels, kernel_size=3, padding=1)

        # Residual blocks
        self.res_block_1 = ResidualBlock(num_conv_channels)
        self.res_block_2 = ResidualBlock(num_conv_channels)
        self.res_block_3 = ResidualBlock(num_conv_channels)
        self.res_block_4 = ResidualBlock(num_conv_channels)

        # Output convolution
        self.output_conv = nn.Conv2d(num_conv_channels, num_bits_per_symbol, kernel_size=3, padding=1)

    def forward(self, inputs):
        y, y2 = inputs

        # Input convolution
        z = self.input_conv(y)

        # Residual blocks
        z = self.res_block_1(z)
        z = self.res_block_2(z)
        z = self.res_block_3(z)
        z = self.res_block_4(z)

        # Output convolution
        z = self.output_conv(z)

        return z

#####################################################################################################################



In [ ]:
model = Model(ResidualBlock, 1)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
#make a training loop
num_epochs = 100

data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Create an instance of the Rx1Model
#model = Model(S, F, Qm)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0

    for pilot_iq, pdsch_iq, labels in data_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(pilot_iq, pdsch_iq)
        
        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

    # Print average loss for the epoch
    average_loss = total_loss / len(data_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

# Save the trained model if needed
torch.save(model.state_dict(), 'rx_model.pth')
